In [1]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

# Run the installation commands
if 'google.colab' in sys.modules:
    print("Running in Google Colab")
    !pip install bitsandbytes accelerate
    !pip install gradio
else:
    print("Not running in Google Colab")
    !pip install transformers accelerate datasets bitsandbytes
    !pip install gradio

# Clean up the notebook
clean_notebook()


Notebook cleaned.


In [2]:
import os
os.environ['HF_TOKEN'] ="your_token_here"
hf_token               = os.environ['HF_TOKEN']

In [3]:


################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False



In [4]:
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
import torch

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model_id = "scb10x/llama-3-typhoon-v1.5-8b-instruct"

model     = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,device_map="auto",token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(model_id) 
   
   


config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [6]:
 
def generate_response(prompt, max_tokens=512, temperature=0.6, top_p=0.9):
    
    messages = [
        {"role": "system", "content": "You are an AI content  writer.Generate a well structured 256 words thai language blog for the given topics"},
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)


    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_tokens,
        pad_token_id=model.config.eos_token_id, 
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)


prompt = "บอกวิธีเลี้ยงแมว 7 ตัว"
output = generate_response(prompt)

print(output)

สวัสดีค่ะ, ในวันนี้เราจะมาบอกวิธีเลี้ยงแมว 7 ตัวให้ได้รับความสุขและความอบอุ่นอย่างเต็มที่

1. จัดเตรียมที่อยู่อาศัย: คุณจำเป็นต้องมีพื้นที่ที่เพียงพอสำหรับแมวทั้ง 7 ตัว โดยที่แต่ละตัวมีพื้นที่ส่วนตัวและพื้นที่ที่สามารถเล่นได้ คุณอาจจะต้องทำห้องแมวหลายห้องเพื่อให้แมวทั้ง 7 ตัวมีพื้นที่ที่เพียงพอ

2. ให้อาหารที่ดี: คุณจำเป็นต้องให้อาหารที่มีคุณภาพสูงแก่แมวทั้ง 7 ตัว เพื่อให้พวกเขาได้รับสารอาหารที่เพียงพอสำหรับการเจริญเติบโตและการทำงานของร่างกาย

3. ให้น้ำสะอาด: แมวทั้ง 7 ตัวของคุณจำเป็นต้องได้รับน้ำสะอาดที่เพียงพอ เพื่อให้ร่างกายของพวกเขาได้รับน้ำที่เพียงพอสำหรับการทำงาน

4. ให้ความรักและการดูแล: แมวทั้ง 7 ตัวของคุณจำเป็นต้องได้รับความรักและการดูแลจากคุณ เพื่อให้พวกเขาได้รับความอบอุ่นและความสุข

5. ให้เวลาในการเล่น: คุณจำเป็นต้องให้เวลาแก่แมวทั้ง 7 ตัวในการเล่น เพื่อให้พวกเขาได้รับความสนุกสนานและความสุข

6. ให้การตรวจสุขภาพ: คุณจำเป็นต้องให้แมวทั้ง 7 ตัวของคุณได้รับการตรวจสุขภาพอย่างสม่ำเสมอ เพื่อให้คุณสามารถตรวจสอบสุขภาพของพวกเขาได้

7. ให้ความสำคัญกับการทำความสะอาด: คุณจำเป็นต้องให้ควา

In [7]:
prompt = "ณัฐกานต์   แปลว่าอะไร"
output = generate_response(prompt,temperature=0.85)

print(output)

คำแปลของ "ณัฐกานต์" คือ "พระจันทร์ที่สว่างสดใส" หรือ "แสงจันทร์ที่สว่างสดใส" เป็นชื่อที่มีความหมายดีและมีความสวยงาม ในภาษาไทย คำนี้มีความหมายถึงความสว่างใสและความสุข.


In [8]:

prompt = "ธนันทิกานต์  แปลว่าอะไร"
output = generate_response(prompt,temperature=0.85)

print(output)

ธนันทิกานต์ เป็นชื่อไทยที่มีความหมายว่า "ผู้ที่มีความสุขและความเจริญรุ่งเรือง" หรือ "ผู้ที่มีชีวิตที่สุขสันต์และมีความสำเร็จ". ชื่อนี้ประกอบด้วยคำ 3 คำ คือ "ธนัน" ซึ่งหมายถึง "ความสุข", "ทิกา" หมายถึง "ความเจริญรุ่งเรือง", และ "กานต์" ซึ่งหมายถึง "ความสำเร็จ". ชื่อธนันทิกานต์จึงสะท้อนถึงความหวังที่ผู้ให้กำเนิดจะให้กับบุตรหรือบุตรสาวของตน.


In [9]:
import gradio as gr


# Example prompts
examples = [
    ["บอกวิธีเลี้ยงแมว 7 ตัว"],
    ["เล่าประวัติของกรุงเทพมหานคร"],
    ["วิธีการทำอาหารไทยยอดนิยม"],
    ["แนะนำสถานที่ท่องเที่ยวในประเทศไทย"],
    ["เขียนบทความเกี่ยวกับการดูแลสุขภาพจิต"]
]

def chat(prompt, temperature):
    return generate_response(prompt, temperature=temperature)

# Create Gradio interface
interface = gr.Interface(
    fn=chat,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your prompt here..."),
        gr.Slider(minimum=0.1, maximum=1.0, step=0.1, value=0.85, label="Temperature")
    ],
    outputs=gr.Textbox(),
    title="Thai Language Blog Generator",
    description="Enter a topic to generate a well-structured 256-word Thai language blog.",
    examples=examples
)

# Launch the interface
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://affd4e62fbf40724f3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
